In [1]:
from model_basic_AUnet import *
from data import *
import os, os.path

vid="v017_5" #version id should match the file number. Last number shows the cross-validation fold number
# Ran on DellWS with GeForce RTX3060 GPU

# Count the number of train and valid files
train_dir = 'mass_seg_08/train0'+vid[-1]+'/mg'
train_count=len([name for name in os.listdir(train_dir) if os.path.isfile(os.path.join(train_dir, name))])

valid_dir = 'mass_seg_08/valid0'+vid[-1]+'/mg'
valid_count=len([name for name in os.listdir(valid_dir) if os.path.isfile(os.path.join(valid_dir, name))])

### Train with data generator

In [2]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger

#Data augmentation
data_gen_args = dict(rotation_range=90,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    shear_range=0.2,
                    zoom_range=0.2,
                    #brightness_range=[0,1],
                    horizontal_flip=True,
                    vertical_flip=True,
                    fill_mode='wrap')

# SETTINGS ***
batch_size=2
learning_rate=1e-4

train_gen = trainGenerator(batch_size,'mass_seg_08/train0'+vid[-1],'mg','mask',data_gen_args,save_to_dir = None)
valid_gen = trainGenerator(batch_size,'mass_seg_08/valid0'+vid[-1],'mg','mask',data_gen_args,save_to_dir = None)

# train_count images are used for training, valid_count images for validating
train_steps = train_count//batch_size
valid_steps = valid_count//batch_size

# SETTINGS ***
loss=dice_loss
steps_per_epoch=3*train_steps
num_epochs=100

model = get_aunet()

opt = tf.keras.optimizers.Adam(learning_rate)
metrics = ["acc", dice_coef]
model.compile(loss=loss, optimizer=opt, metrics=metrics)

callbacks = [
             ModelCheckpoint('files_mass_seg_xval/unet_mass_seg_'+vid+'.hdf5', verbose=1, save_best_model=True),
             ReduceLROnPlateau(monitor="val_loss", patience=3, factor=0.1, verbose=1, min_lr=1e-8),
             CSVLogger("files_mass_seg_xval/data_"+vid+".csv"),
             EarlyStopping(monitor="val_loss", patience=5, verbose=1)
            ]

model.fit(train_gen, validation_data=valid_gen, steps_per_epoch=steps_per_epoch, validation_steps=valid_steps, 
                    epochs=num_epochs, callbacks=callbacks)

Found 283 images belonging to 1 classes.
Found 283 images belonging to 1 classes.
Epoch 1/100
423/423 [==============================] - ETA: 0s - loss: 0.7626 - acc: 0.9323 - dice_coef: 0.2374Found 69 images belonging to 1 classes.
Found 69 images belonging to 1 classes.

Epoch 1: saving model to files_mass_seg_xval\unet_mass_seg_v017_5.hdf5
423/423 [==============================] - 218s 448ms/step - loss: 0.7626 - acc: 0.9323 - dice_coef: 0.2374 - val_loss: 0.6663 - val_acc: 0.9772 - val_dice_coef: 0.3337 - lr: 1.0000e-04
Epoch 2/100
423/423 [==============================] - ETA: 0s - loss: 0.7168 - acc: 0.9716 - dice_coef: 0.2829
Epoch 2: saving model to files_mass_seg_xval\unet_mass_seg_v017_5.hdf5
423/423 [==============================] - 183s 432ms/step - loss: 0.7168 - acc: 0.9716 - dice_coef: 0.2829 - val_loss: 0.7403 - val_acc: 0.9784 - val_dice_coef: 0.2597 - lr: 1.0000e-04
Epoch 3/100
423/423 [==============================] - ETA: 0s - loss: 0.7359 - acc: 0.9654 - dice_c

### validate your model and save predicted results

In [3]:
validGene = testGenerator("mass_seg_08/valid0"+vid[-1]+"/pred",num_image=valid_count)
model.load_weights("files_mass_seg_xval/unet_mass_seg_"+vid+".hdf5")
results = model.predict(validGene,valid_count,verbose=1)
saveResult("mass_seg_08/valid0"+vid[-1]+"/pred",results,vid[:4])

69/69 [==============================] - 18s 255ms/step


D:\research\breastCancer\mass_seg\codes\data.py:110: UserWarning: mass_seg_08/valid05/pred\1_predict_v017.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict_%s.png"%(i,vid)),img)
D:\research\breastCancer\mass_seg\codes\data.py:110: UserWarning: mass_seg_08/valid05/pred\4_predict_v017.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict_%s.png"%(i,vid)),img)
D:\research\breastCancer\mass_seg\codes\data.py:110: UserWarning: mass_seg_08/valid05/pred\5_predict_v017.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict_%s.png"%(i,vid)),img)
D:\research\breastCancer\mass_seg\codes\data.py:110: UserWarning: mass_seg_08/valid05/pred\9_predict_v017.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict_%s.png"%(i,vid)),img)
D:\research\breastCancer\mass_seg\codes\data.py:110: UserWarning: mass_seg_08/valid05/pred\10_predict_v017.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict_%s.png

In [4]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 640, 640, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 640, 640, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 640, 640, 32  128        ['conv2d[0][0]']                 
 alization)                     )                                                             

                                                                  'conv2d_11[0][0]']              
                                                                                                  
 activation_1 (Activation)      (None, 40, 40, 256)  0           ['add[0][0]']                    
                                                                                                  
 conv2d_13 (Conv2D)             (None, 40, 40, 1)    257         ['activation_1[0][0]']           
                                                                                                  
 activation_2 (Activation)      (None, 40, 40, 1)    0           ['conv2d_13[0][0]']              
                                                                                                  
 up_sampling2d (UpSampling2D)   (None, 80, 80, 1)    0           ['activation_2[0][0]']           
                                                                                                  
 lambda (L

 ormalization)                  6)                                                                
                                                                                                  
 activation_6 (Activation)      (None, 160, 160, 25  0           ['batch_normalization_12[0][0]'] 
                                6)                                                                
                                                                                                  
 conv2d_26 (Conv2D)             (None, 160, 160, 64  16448       ['activation_6[0][0]']           
                                )                                                                 
                                                                                                  
 conv2d_transpose_4 (Conv2DTran  (None, 160, 160, 64  36928      ['conv2d_26[0][0]']              
 spose)                         )                                                                 
          

 multiply_3 (Multiply)          (None, 640, 640, 32  0           ['lambda_3[0][0]',               
                                )                                 'batch_normalization_1[0][0]']  
                                                                                                  
 conv2d_35 (Conv2D)             (None, 640, 640, 32  1056        ['multiply_3[0][0]']             
                                )                                                                 
                                                                                                  
 conv2d_transpose_7 (Conv2DTran  (None, 640, 640, 32  18464      ['conv2d_30[0][0]']              
 spose)                         )                                                                 
                                                                                                  
 batch_normalization_15 (BatchN  (None, 640, 640, 32  128        ['conv2d_35[0][0]']              
 ormalizat